In [125]:
%pylab inline

import pandas as pd
import seaborn as sns
sns.set()
import pickle
from sklearn.neighbors import KNeighborsClassifier , RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split , cross_val_score, StratifiedShuffleSplit, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, LabelEncoder
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import SequentialFeatureSelector

Populating the interactive namespace from numpy and matplotlib


In [12]:
with open('atxac_de.pickle', 'rb') as read_aacde:
    atxac_df = pickle.load(read_aacde)

In [54]:
X = atxac_df.drop(columns=['Outcome_Type','Updated_Outcome_Type','Adoption'])
y = atxac_df['Adoption']

In [55]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=.2, random_state=88,stratify=y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=24,stratify=y_train_val)

In [5]:
#Stand. - 'AgeYrs' , 'TimeSinceIntake'
num_features = ['AgeYrs' , 'TimeSinceIntake']
#OHE - 'Sex','Spayed_Neutered','Sex_Upon_Outcome', 'OutcomeBreed' , 'OutcomeColor','Intake_Type','Intake_Condition','IntakeMonth','No_Known_Owner'
cate_features = ['Sex','Spayed_Neutered','Sex_Upon_Outcome', 'OutcomeBreed' , 'OutcomeColor','Intake_Type','Intake_Condition','IntakeMonth','No_Known_Owner']
combined = num_features+cate_features

In [27]:
atxac_df[combined].dtypes

AgeYrs              float64
TimeSinceIntake     float64
Sex                  object
Spayed_Neutered      object
Sex_Upon_Outcome     object
OutcomeBreed         object
OutcomeColor         object
Intake_Type          object
Intake_Condition     object
IntakeMonth           int64
No_Known_Owner      float64
dtype: object

In [22]:
atxac_df['TimeSinceIntake'] = atxac_df['TimeSinceIntake'].dt.days.astype(float)

In [100]:
atxac_df['TimeSinceIntake'].head()

0     5.0
1     0.0
2     0.0
3    66.0
4     4.0
Name: TimeSinceIntake, dtype: float64

In [33]:
coltransf = make_column_transformer((StandardScaler(),num_features),(OneHotEncoder(handle_unknown='ignore'),cate_features),remainder='passthrough')

In [51]:
stdsc = StandardScaler()
stdsc.fit(X_train_val['AgeYrs','TimeSinceIntake'])

KeyError: ('AgeYrs', 'TimeSinceIntake')

In [56]:
X_train_val[num_features]

,AgeYrs,TimeSinceIntake
14594,0.019231,0.0
20820,0.083333,4.0
1106,0.019231,0.0
16331,0.166667,42.0
3122,0.038462,0.0
...,...,...
21259,0.083333,1.0
28813,0.083333,28.0
7953,2.000000,6.0
37167,0.019231,0.0


In [108]:
knn = KNeighborsClassifier()
logreg = LogisticRegression(max_iter=10000000)
bernnb = BernoulliNB()
dtree = DecisionTreeClassifier()

In [133]:
pipe1 = make_pipeline(coltransf, knn)
pipe2 = make_pipeline(coltransf, logreg)
pipe3 = make_pipeline(coltransf, sfs1, bernnb)
pipe4 = make_pipeline(coltransf, dtree)

In [95]:
y_pred1 = cross_val_predict(pipe1, X_train_val[combined], y_train_val)

In [47]:
y_pred2 = cross_val_predict(pipe2, X_train_val, y_train_val)

/Users/chuckcao/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/chuckcao/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_m

In [134]:
y_pred3 = cross_val_predict(pipe3, X_train_val[cate_features], y_train_val)

ValueError: pos_label=1 is invalid. Set it to a label in y_true.

In [110]:
y_pred4 = cross_val_predict(pipe4, X_train_val[combined], y_train_val)

In [59]:
lrcv = LogisticRegressionCV(max_iter=10000000,random_state=0)

In [58]:
X_trtr = coltransf.fit_transform(X_train_val[combined])

In [60]:
lrcv.fit(X_trtr,y_train_val)

LogisticRegressionCV(max_iter=10000000, random_state=0)

In [62]:
lrcv.coef_

array([[-4.93931694e-01, -4.30502481e-01,  9.28196764e-01,
         8.56293393e-01, -8.52530804e-01,  3.87473878e-01,
         4.68819515e-01, -8.99321997e-01, -8.81405572e-01,
         9.28196764e-01, -5.99155199e-03,  3.05882763e-04,
        -1.37266792e-03, -3.36060760e-03, -1.00031392e-03,
        -1.87847498e-02,  6.47341776e-02, -1.62536181e-03,
         9.18738387e-03,  3.59639877e-03, -3.19998496e-03,
        -2.01753414e-02, -9.12080257e-03,  2.17594158e-03,
        -1.83427264e-03, -8.62997443e-03, -4.11739357e-03,
        -3.60379689e-03, -4.84676751e-03, -7.70587206e-03,
        -2.66106450e-08,  1.70965877e-03,  2.77672272e-04,
         3.35180923e-03,  4.86686849e-03,  5.06343311e-04,
        -1.92849733e-03, -7.98979031e-02, -4.89741242e-02,
         1.43046694e-04, -4.27999880e-03,  1.35714999e-03,
         5.62017979e-03, -3.31824248e-02,  6.36509421e-02,
         3.59598530e-03,  5.30217042e-02,  2.02454709e-01,
        -7.65623993e-04,  9.70246601e-05,  5.51093567e-0

In [67]:
y_pred5 = lrcv.predict(X_trtr)

In [68]:
print("LogRegCV confusion matrix: \n\n", metrics.confusion_matrix(y_train_val, y_pred5,labels=['Adoption','Not Adopted']))

LogRegCV confusion matrix: 

 [[13150  1494]
 [ 3110 15918]]


In [94]:
print("kNN confusion matrix: \n\n", metrics.confusion_matrix(y_train_val, y_pred1,labels=['Adoption','Not Adopted']))

kNN confusion matrix: 

 [[13041  1603]
 [ 2600 16428]]


In [51]:
print("LogReg confusion matrix: \n\n", metrics.confusion_matrix(y_train_val, y_pred2,labels=['Adoption','Not Adopted']))

LogReg confusion matrix: 

 [[17860  1631]
 [ 6312 15222]]


In [93]:
print("BNB1 confusion matrix: \n\n", metrics.confusion_matrix(y_train_val, y_pred3,labels=['Adoption','Not Adopted']))

BNB confusion matrix: 

 [[13815   829]
 [ 4285 14743]]


In [98]:
print("BNB2 confusion matrix: \n\n", metrics.confusion_matrix(y_train_val, y_pred3,labels=['Adoption','Not Adopted']))

BNB2 confusion matrix: 

 [[13754   890]
 [ 4999 14029]]


In [111]:
print("Decision Tree confusion matrix: \n\n", metrics.confusion_matrix(y_train_val, y_pred4,labels=['Adoption','Not Adopted']))

Decision Tree confusion matrix: 

 [[12350  2294]
 [ 2247 16781]]


In [113]:
dtree.feature_importances_

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [114]:
X_trtr.shape

(33672, 384)

In [130]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X_train_val[cate_features])
X_tvtr = ohe.transform(X_train_val[cate_features]).toarray()

In [131]:
sfs1 = SequentialFeatureSelector(bernnb,scoring='average_precision')

In [132]:
sfs1.fit(X_tvtr,y_train_val)

ValueError: pos_label=1 is invalid. Set it to a label in y_true.